In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import joblib


In [6]:
# Define the file path
file_path = 'NIFTY50_all.csv'

# Load the dataset
df = pd.read_csv(file_path)



In [7]:
df.head()

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.0,959.0,962.90,984.72,27294366,2.687719e+15,NaN,9859619.0,0.3612
1,2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.0,885.0,893.90,941.38,4581338,4.312765e+14,NaN,1453278.0,0.3172
2,2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.0,887.0,884.20,888.09,5124121,4.550658e+14,NaN,1069678.0,0.2088
3,2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.0,929.0,921.55,929.17,4609762,4.283257e+14,NaN,1260913.0,0.2735
4,2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.0,980.0,969.30,965.65,2977470,2.875200e+14,NaN,816123.0,0.2741


In [8]:
corr_matrix = df.select_dtypes(include=['number']).corr()
corr_matrix

,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Prev Close,1.000000,0.999718,0.999616,0.999601,0.999505,0.999518,0.999622,-0.118742,0.052888,-0.149562,-0.132987,-0.009034
Open,0.999718,1.000000,0.999857,0.999863,0.999748,0.999761,0.999867,-0.118779,0.052704,-0.149840,-0.133025,-0.009078
High,0.999616,0.999857,1.000000,0.999790,0.999881,0.999892,0.999937,-0.118595,0.053697,-0.148860,-0.132979,-0.009962
Low,0.999601,0.999863,0.999790,1.000000,0.999868,0.999878,0.999929,-0.118914,0.052144,-0.150774,-0.133141,-0.008543
Last,0.999505,0.999748,0.999881,0.999868,1.000000,0.999991,0.999950,-0.118713,0.053110,-0.149736,-0.133051,-0.009367
Close,0.999518,0.999761,0.999892,0.999878,0.999991,1.000000,0.999963,-0.118705,0.053106,-0.149751,-0.133042,-0.009362
VWAP,0.999622,0.999867,0.999937,0.999929,0.999950,0.999963,1.000000,-0.118757,0.053013,-0.149787,-0.133096,-0.009425
Volume,-0.118742,-0.118779,-0.118595,-0.118914,-0.118713,-0.118705,-0.118757,1.000000,0.629582,0.703770,0.829507,-0.209112
Turnover,0.052888,0.052704,0.053697,0.052144,0.053110,0.053106,0.053013,0.629582,1.000000,0.840716,0.573714,-0.194729
Trades,-0.149562,-0.149840,-0.148860,-0.150774,-0.149736,-0.149751,-0.149787,0.703770,0.840716,1.000000,0.567846,-0.252629


In [9]:
df.describe()

,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
count,235192.000000,235192.000000,235192.000000,235192.000000,235192.000000,235192.000000,235192.00000,2.351920e+05,2.351920e+05,1.203440e+05,2.191150e+05,219115.000000
mean,1266.196349,1267.759708,1286.581440,1247.488465,1266.388302,1266.554351,1267.13230,3.045903e+06,1.610138e+14,6.196427e+04,1.315098e+06,0.502997
std,2581.370320,2585.259609,2619.649216,2546.621396,2581.392543,2582.140942,2582.69998,7.333981e+06,3.298085e+14,6.866457e+04,2.831670e+06,0.190019
min,0.000000,8.500000,9.750000,8.500000,9.100000,9.150000,9.21000,3.000000e+00,1.047000e+07,1.100000e+01,5.000000e+00,0.023600
25%,274.300000,275.000000,279.500000,269.600000,274.400000,274.350000,274.69750,2.190095e+05,1.612816e+13,2.183400e+04,1.253830e+05,0.364700
50%,566.500000,567.025000,576.900000,556.500000,567.000000,566.700000,566.94000,1.010938e+06,6.832603e+13,4.406800e+04,5.017560e+05,0.511000
75%,1242.200000,1243.312500,1263.000000,1221.650000,1242.900000,1242.400000,1242.66250,3.019851e+06,1.863835e+14,7.893550e+04,1.452233e+06,0.638400
max,32861.950000,33399.950000,33480.000000,32468.100000,32849.000000,32861.950000,32975.24000,4.810589e+08,3.564334e+16,1.643015e+06,2.325307e+08,1.000000


In [10]:
# 1. Define Features (X) and Target (y) after checking corelation between close and other column 

features = ['Open', 'High', 'Low', 'Prev Close','VWAP']
target = ['Close']

X = df[features]
y = df[target]

In [11]:
# 2. Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
model = LinearRegression()
print("Training model...")
model.fit(X_train, y_train)
print("Model training complete.")

Training model...
Model training complete.


In [13]:
# 4. Make Predictions on the Test Set
y_pred = model.predict(X_test)


In [14]:
# 5. Evaluate the Model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


In [15]:
print("\n--- Model Evaluation ---")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R²) Score: {r2}")


--- Model Evaluation ---
Mean Squared Error (MSE): 370.3286249835853
R-squared (R²) Score: 0.9999452719180613


In [16]:
model_filename = 'nifty50model.joblib'
joblib.dump(model, model_filename)


['nifty50model.joblib']